#Creating a Data Warehouse Through Joins and Unions
The dataset you'll use is an ecommerce dataset that has millions of Google Analytics records for the Google Merchandise Store loaded into BigQuery

**Scenario**:Your marketing team provided you and your data science team all of the product reviews for your ecommerce website. You partner with them to create a data warehouse in BigQuery which joins together data from three sources:

* Website ecommerce data
* Product inventory stock levels and lead times
* Product review sentiment analysis

In this lab, you examine a new dataset based on product reviews.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Colab includes the google.colab.data_table package that can be used to display large pandas dataframes as an interactive data table. It can be enabled with:

In [4]:
%load_ext google.colab.data_table

##Explore the product sentiment dataset

In [5]:
%%bigquery --project bigquery-288322
#standardSQL
SELECT
  SKU,
  name,
  sentimentScore,
  sentimentMagnitude
FROM
  `data-to-insights.ecommerce.products`
ORDER BY
  sentimentScore DESC
LIMIT 5

,SKU,name,sentimentScore,sentimentMagnitude
0,GGOBJGOWUSG69402,USB wired soundbar - in store only,1.0,1.0
1,GGOEGOAR013099,Stylus Pen w/ LED Light,0.9,1.4
2,GGOEGADJ056816,Men's Watershed Full Zip Hoodie Grey,0.9,1.4
3,GGOEGOAB021499,Metal Texture Roller Pen,0.9,1.4
4,GGOEGEVB070599,G Noise-reducing Bluetooth Headphones,0.9,0.5


##Join datasets to find insights

### 1.Calculate daily sales volume by productSKU

In [6]:
%%bigquery --project bigquery-288322
# pull what sold on 08/01/2017
#CREATE OR REPLACE TABLE ecommerce.sales_by_sku_20170801 AS
SELECT DISTINCT
  productSKU,
  SUM(IFNULL(productQuantity,0)) AS total_ordered
FROM
  `data-to-insights.ecommerce.all_sessions_raw`
WHERE date = '20170801'
GROUP BY productSKU
ORDER BY total_ordered DESC #462 skus sold

,productSKU,total_ordered
0,GGOEGOAQ012899,456
1,GGOEGDHC074099,334
2,GGOEGOCB017499,319
3,GGOEGOCC077999,290
4,GGOEGFYQ016599,253
...,...,...
457,9182726,0
458,9182173,0
459,9182752,0
460,9180862,0


### 2.Join sales data and inventory data
Using a join, enrich the website ecommerce data with the following fields from the product inventory dataset:

name,
stockLevel,
restockingLeadTime,
sentimentScore,
sentimentMagnitude,

In [8]:
%%bigquery --project bigquery-288322
# join against product inventory to get name
SELECT DISTINCT
  website.productSKU,
  website.total_ordered,
  inventory.name,
  inventory.stockLevel,
  inventory.restockingLeadTime,
  inventory.sentimentScore,
  inventory.sentimentMagnitude
FROM
  (SELECT DISTINCT
  productSKU,
  SUM(IFNULL(productQuantity,0)) AS total_ordered
  FROM
    `data-to-insights.ecommerce.all_sessions_raw`
  WHERE date = '20170801'
  GROUP BY productSKU) AS website
LEFT JOIN `data-to-insights.ecommerce.products` AS inventory
ON website.productSKU = inventory.SKU
ORDER BY total_ordered DESC

,productSKU,total_ordered,name,stockLevel,restockingLeadTime,sentimentScore,sentimentMagnitude
0,GGOEGOAQ012899,456,Ballpoint LED Light Pen,2098.0,11.0,0.4,0.7
1,GGOEGDHC074099,334,17oz Stainless Steel Sport Bottle,1390.0,13.0,0.8,1.3
2,GGOEGOCB017499,319,Leatherette Journal,4978.0,36.0,0.5,0.9
3,GGOEGOCC077999,290,Spiral Journal with Pen,4668.0,10.0,0.1,0.3
4,GGOEGFYQ016599,253,Foam Can and Bottle Cooler,4495.0,10.0,0.7,1.2
...,...,...,...,...,...,...,...
457,9182173,0,Stylus Pen w/ LED Light,0.0,11.0,0.2,0.4
458,9182752,0,Men's Short Sleeve Performance Badge Tee Char...,0.0,19.0,0.4,0.7
459,GGOEGAAX0339,0,Men's Vintage Badge Tee White,210.0,9.0,0.1,0.5
460,9182182,0,None,NaN,NaN,NaN,NaN
